## Métrica desconocimiento

In [43]:
from dbconn import querydb, querydbtopandas 
import pandas as pd 

In [44]:
mayor = "2020-05"
menor = "2019-04"

In [51]:
clinicas_propias_id = pd.read_excel("centros_propios_swiss.xlsx",index_col=0).id_pre_prestador.to_list()


In [45]:
data = querydbtopandas(f"""select    a11.id_encuesta  id_encuesta,    
                a11.id_afiliado  id_afi_afiliado,
                a11.id_prestador  id_pre_prestador,                   
                max(a13.desc_pre_nombre)  desc_pre_nombre,
                pre_domicilio pre_domicilio,
                a14.desc_lugar_domi,
                max(a13.vigente)  vigente,
                a11.id_transac_tipo  id_transac_tipo,            
                a11.id_prest_prestacion  id_pres_prestacion,
                max(a18.desc_pres_prestacion)  desc_pres_prestacion,
                max(a18.prestac_pres_prestacion)  prestac_pres_prestacion,
                a11.fec_encuesta,
                sum(distinct a11.i_CNT_desc)  QDESCONOCIDAS
from      DBA.ft_prest_encuestas              a11
                join        DBA.d_pre_prestador   a13
                  on        (a11.id_prestador = a13.id_pre_prestador)
                join        DBA.d_pres_prestacion a18
                  on        (a11.id_prest_prestacion = a18.id_pres_prestacion)
                left join DBA.d_pre_prestador_consultorio a14
                  on        (a13.desc_pre_nombre = a14.pre_nombre)
                  
where (cast(a11.fec_encuesta as date) >= date("{menor}") and cast(a11.fec_encuesta as date) < date("{mayor}")
and a11.id_moti_desconoc <> 1
and a11.id_prestador <> 0)
group by             a11.id_encuesta,
                a11.fec_encuesta,
                a14.desc_lugar_domi,
                a11.id_afiliado,
                a11.id_prestador, 
                a13.pre_domicilio,
                a13.id_pre_especialidad,
                a11.id_transac_tipo,
                a11.id_prest_prestacion,
                DATE(a11.fec_consumo),
                a11.id_tie_mes_c""")

In [46]:
data.fec_encuesta = data.fec_encuesta.astype(str).str[:7]

In [47]:
data.fec_encuesta.nunique()

13

In [48]:
for i in data.fec_encuesta.unique():
    df = data[data["fec_encuesta"] == i]
    
    
    
    cma = df[df["id_pre_prestador"].isin(clinicas_propias_id)][["id_pre_prestador", "desc_pre_nombre"]].rename(columns={"desc_pre_nombre": "cma"})
    df = pd.merge(df, cma, how="outer").fillna("No aplica")

    #count de encuestas
    encuestas_desconocidas = df.groupby(["id_pre_prestador","desc_pre_nombre","cma"])["id_encuesta"].nunique()

    #count de afiliados
    afiliados_desconocidos = df.groupby(["id_pre_prestador","desc_pre_nombre", "cma"])["id_afi_afiliado"].nunique()

    df = pd.concat([encuestas_desconocidas, afiliados_desconocidos],axis=1).reset_index()

    df.rename(columns = {"id_pre_prestador":"Prestador",
                        "desc_pre_nombre":"Razón Social",
                        "id_encuesta":"Q desconocidas",
                        "id_afi_afiliado":"Q afiliados"},inplace=True)

    df = df[['Prestador', 'Razón Social', 'cma', 'Q afiliados', 'Q desconocidas']]
    final = df.sort_values(by= "Q afiliados", ascending = False)
    final.to_parquet(f'datos_desconocimiento/{i}.gzip',compression='gzip') 


In [49]:
final

,Prestador,Razón Social,cma,Q afiliados,Q desconocidas
116,37047,Asociación Médica de Rosario,No aplica,18,27
142,52769,Colegio de Kinesiólogos de la Pcia. de Bs. As....,No aplica,14,30
8,1127,Hospital Británico,No aplica,12,15
34,2085,Clínica San Camilo,No aplica,12,23
28,1632,SWISS MEDICAL CENTER BARRIO NORTE,SWISS MEDICAL CENTER BARRIO NORTE,12,12
...,...,...,...,...,...
183,80614,"Lodeiro,Jorge L.",No aplica,1,1
184,80617,"Kuperman,Mirta A.",No aplica,1,1
46,22131,"Hofman,Elías",No aplica,1,1
186,80776,"Tau Anzoategui,Pablo",No aplica,1,1
